In [1]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import pandas as pd
import vaex

import numpy as np
from scipy.stats import multinomial, norm

from IPython.display import clear_output

# Functions

In [2]:
def search_4_invalid(df, val=50):
    # searches for columns with highest scores of missing/NaN values
    
    # Gathering missing/NaNs for every column
    invalid = []
    cols = df.get_column_names()
    
    for count, col in enumerate(cols):
        invalid.append(
            df[col]\
            .isna()\
            .sum()\
            .item()\
        )
        clear_output(wait=True)
        print(f'{count+1}/{len(cols)} columns done.')
        
    print('All done!')
    
    # Organizing gathered values in a dataframe
    miss = pd.DataFrame(
        data=np.array(invalid), 
        index=cols, 
        columns=['invalids']
    )\
        .sort_values(by='invalids', ascending=False
    )
    
    # Format the count column as a percentage of the original dataframe's length
    miss['invalids'] = round(miss['invalids']*100/df.shape[0], 2).astype('str') + '%'
    
    # Displaying large values
    query = miss.loc[
            np.ceil((miss['invalids'].str.rstrip('%').astype('float'))) > val
    ]
    
    display(query)
    
    # Get columns with biggest score (index from filtered dataframe)
    bad_cols_list = list(query.index)
    
    return bad_cols_list


def clear(df_in, bad_cols_list):
    # Deleting columns comprised mostly by invalid values
    df_out = df_in.drop(bad_cols_list)
    print(f'Dropped {df_in.shape[1] - df_out.shape[1]} columns. {df_out.shape[1]} left.')
    return df_out


def column_selector(df, original_col):
    
    all_cols = df.get_column_names()
    selection = [agg_col for agg_col in all_cols if agg_col.startswith(original_col)]
    
    return selection


def fix_outliers(df_in):
    """Find outliers by going through the columns and replacing them with the 10th or 90th percentiles, 
    depending on their values. The outliers were defined as above or below the expression (mean(+-)3stddev)
    for the column.
    
    Args:
        df_in (pd.DataFrame)
    Returns:
        df_out (pd.DataFrame)
    """
    
    df_out = df_in.copy()
    for count, (col, dtype) in enumerate(zip(df_out.columns, df_out.dtypes.values)):
        if (str(dtype).startswith('float') or str(dtype).startswith('int')):
            stddev = df_out[col].std()
            mean = df_out[col].mean()
            if (df_out[col].min() < (mean-3*stddev)) or (df_out[col].max() > (mean+3*stddev)):
                perc_10 = df_out[col].quantile(0.10)
                perc_90 = df_out[col].quantile(0.90)
                df_out.loc[df_out[col] < (mean-3*stddev), col] = perc_10
                df_out.loc[df_out[col] > (mean+3*stddev), col] = perc_90       
        clear_output(wait=True)
        print(f'{count+1}/{len(df_out.columns)} columns done.')
        
    print('All done!')          
    return df_out


def fill_numerical_invalids(df_in, cols=None):
    # Skip column in case there are no values to change
    if cols is None:
        cols = df_in.columns
    
    df_out = df_in.copy()
    for count, col in enumerate(cols):
        
        if df_out[col].isna().sum() == 0:
            clear_output(wait=True)
            print(f'{count+1}/{len(cols)} columns done.')
            continue
        # Generate a [scipy] normal distribution from current column's values
        norm_dist = norm(
            loc=df_out[col].mean(),
            scale=df_out[col].std()
        )
        # Series made of values pulled from said distribution and length equal to the amount of invalids
        values = pd.Series(
            data=norm.rvs(size=(len(df_out),))
        )
        # Fill column's missing values with those from previously generated normal distribution
        df_out.loc[df_out[col].isna(), col] = values
        
        clear_output(wait=True)
        print(f'{count+1}/{len(cols)} columns done.')
    
    
    print('All done!')
    return df_out


def fill_categorical_invalids(df_in, categoricals):
    """Fill the input dataframe with values taken from a multinomial bernoulli distribution built from the
    column being filled.
    
    Args:
        df_in (pd.DataFrame): input dataframe
        categoricals (array): names of categorical columns currently on the dataframe
        
    Returns:
        df_out (pd.DataFrame): output dataframe with no missing/NaN values
    """
    
    df_out = df_in.copy()
    
    for count, col in enumerate(categoricals):
        
        vc = df_out[col].value_counts()
        
        # In case there are no missing values, skip the column
        if df_out[col].isna().sum() == 0:
            clear_output(wait=True)
            print(f'{count+1}/{len(categoricals)} columns done.')
            continue
        # Gathering values to set up a multinomial distribution
        non_missing_index = [ind for ind in vc.index if ind != 'missing' and str(ind) not in ['nan', 'NaN']]
        non_missing_count = [count for count, ind in zip(vc.values, vc.index) if ind != 'miing' and str(ind) not in ['nan', 'NaN']]
        non_missing_probabilities = non_missing_count/sum(non_missing_count)
        # Multinomial distribution object
        dist = multinomial(
            n=1,  # Each call to the multinomial distribution takes 1 try only
            p=non_missing_probabilities
        )

        # Column with indexes pulled randomly from the distribution then mapping back to original values
        indexes = np.argmax(dist.rvs(size=(len(df_out),)), axis=1)
        mapping = dict(
                zip(range(len(non_missing_index)), non_missing_index)
        )
        values = pd.Series(np.array(
            [mapping[index] for index in indexes]
        ).reshape(-1,))
        # Fill missing values with random ones pulled from aforementioned dist;
        df_out.loc[df_out[col].isna(), col] = values
            
    return df_out


def cat_datatypes_check(df_in, categorical_features):
    
    # Shallow copy not to modify input dataframe directly
    df_out = df_in.copy()
    
    # Scan for possible convertions to 'int' and 'str', acceptable types for categorical features
    for feature, dtype in zip(categorical_features, df_out[categorical_features].dtypes.values):
        if str(dtype).startswith('float'):
            df_out[feature] = df_out[feature].astype('int32')
        elif str(dtype).startswith('object'):
            df_out[feature] = df_out[feature].astype('str')
        else:
            continue
            
    return df_out

##### Sequence:
- Load
- Delete mostly invalid features
- Detect outliers and map them to median
- Fill rest of the invalid values on numeric columns
- Fill rest of the invalid values on categorical columns
- Export

# Load

In [3]:
# https://www.kaggle.com/datasets/huseyincot/amex-agg-data-pickle
train = pd.read_pickle('train_agg.pkl', compression='gzip')
test = pd.read_pickle('test_agg.pkl', compression='gzip')

In [4]:
train = vaex.from_pandas(train)
test = vaex.from_pandas(test)

print(f'Train data shape: {train.shape}.')
print(f'Test data shape: {test.shape}.')

Train data shape: (458913, 919).
Test data shape: (924621, 918).


In [5]:
for col, dtype in zip(train.columns, train.dtypes.values):
    if str(dtype).startswith('float'):
        train[col] = train[col].astype('float32')
    elif str(dtype).startswith('int'):
        train[col] = train[col].astype('int32')
        
for col, dtype in zip(test.columns, test.dtypes.values):
    if str(dtype).startswith('float'):
        test[col] = test[col].astype('float32')
    elif str(dtype).startswith('int'):
        train[col] = train[col].astype('int32')

# Invalid entries

In [6]:
%%time
train = clear(train, search_4_invalid(train, val=98))

919/919 columns done.
All done!


,invalids
D_87_std,99.9%
D_87_last,99.82%
D_87_max,99.82%
D_87_min,99.82%
D_87_mean,99.82%
D_88_std,99.57%
D_88_min,99.47%
D_88_max,99.47%
D_88_last,99.47%
D_88_mean,99.47%


Dropped 36 columns. 883 left.
CPU times: total: 1min 56s
Wall time: 1min 53s


In [7]:
# Delete the same columns from the test set
dropped_columns = list(set(test.get_column_names()) - set(train.get_column_names()))
test = test.drop(columns=dropped_columns)

print(test.shape)

(924621, 882)


In [8]:
# Columnn names lists for easier selection later on

# All columns
all_columns = list(train.get_column_names())
# Training features
training_features = list(set(all_columns) - set(['target']))
# Categorical features (as per https://www.kaggle.com/competitions/amex-default-prediction/data)
categorical_features_old = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64',
                        'D_66', 'D_68']
categorical_features = []

for old_cat in categorical_features_old:
    categorical_features += column_selector(train, old_cat)

# Numerical features
numerical_features = list(set(training_features) - set(categorical_features))

# Outliers

In [9]:
%%time
# Outlier treatment function (returns a pandas dataframe)
train = fix_outliers(train.to_pandas_df())

883/883 columns done.
All done!
CPU times: total: 47.1 s
Wall time: 42.9 s


In [10]:
test = fix_outliers(test.to_pandas_df())

882/882 columns done.
All done!


# Filling numerical features 

In [11]:
%%time
# Training set
train = fill_numerical_invalids(
    train, 
    cols=numerical_features
)
# Test set
test = fill_numerical_invalids(
    test,
    cols=numerical_features
)

849/849 columns done.
All done!
CPU times: total: 1min
Wall time: 1min


# Filling categorical features

In [12]:
%%time
# Fill the invalid values on categorical features
train = fill_categorical_invalids(train, categorical_features)
# Make sure dtypes on categorical features are all 'int' or 'str'
train = cat_datatypes_check(train, categorical_features)

# Same process for test set}
test = fill_categorical_invalids(test, categorical_features)
test = cat_datatypes_check(test, categorical_features)

33/33 columns done.
CPU times: total: 29.2 s
Wall time: 29.5 s


# Export

In [13]:
# Export resulting modified dataset
train.to_parquet('train_agg_filtered_2.parquet')
test.to_parquet('test_agg_filtered_2.parquet')

In [15]:
columns = []
maxim = []
minim = []

for col, coltype in zip(numerical_features, train[numerical_features].dtypes.values):
    if str(coltype).startswith('int') or str(coltype).startswith('float'):
        columns.append(col)
        maxim.append(train[col].max())
        minim.append(train[col].min())

data = np.array([columns, maxim, minim]).transpose()
df_minmax = pd.DataFrame(data=data, columns=['columns', 'maxi', 'mini'])
df_minmax.maxi = df_minmax.maxi.astype('float')
df_minmax.mini = df_minmax.mini.astype('float')
df_minmax.sort_values(by='maxi')

,columns,maxi,mini
35,R_23_min,0.003967,0.000000e+00
129,S_19_min,0.004066,0.000000e+00
348,R_18_min,0.004463,0.000000e+00
391,R_17_min,0.006226,0.000000e+00
658,R_13_min,0.008453,0.000000e+00
...,...,...,...
91,R_14_last,35.187500,0.000000e+00
349,B_10_max,43.875000,-2.956390e-03
263,B_40_max,45.906250,6.258488e-06
75,R_14_max,57.531250,6.556511e-07
